In [5]:
import traceback
from typing import (
    List,
    Dict
)
from functools import partial
from itertools import product

import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from tqdm.auto import tqdm

from gc4eptn import dataloaders
from gc4eptn.utils import norms
from gc4eptn.gsp import single_gsp_exp
from gc4eptn.utils.utils import (
    get_module_root, 
    get_dataloader,
    to_adjacency,
    exp_log_grid,
    build_experiment_path,
)

from gc4eptn.utils.metrics import (
    graph_fscore,
    graph_fbscore,
    graph_precision,
    graph_recall,
    hamming_distance,
    ebic,
)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
fstandardize = partial(
    norms.feature_norm,
    norm_fn=norms.standardize,
    features=['V_mag', 'V_ang', 'I_ang', 'I_mag']
)

In [7]:
metrics = {
    'hd': dict(
        fn=hamming_distance,
        type='lowest'
    ),
    'f-score': dict(
        fn=graph_fscore,
        type='highest'
    ),
    'fb-score': dict(
        fn=partial(graph_fbscore, beta=2),
        type='highest'
    ),
    'precision': dict(
        fn=graph_precision,
        type='highest'
    ),
    'recall': dict(
        fn=graph_recall,
        type='highest'
    ),
}

# Multi-Experiments

In [8]:
def multi_gsp_exps(
    dataset_type: str = 'rtds',
    dataset_versions: List[str] = ['v5'],
    difficulties: List[str] = ['novice'],
    topologies = ['complete', 'partial-left', 'partial-right'],
    load_types: List[str] = ['high', 'medium', 'low'],
    n_samples: int = [None],
    metrics: Dict = metrics,
    best_metric_names: List[str] = ['fb-score'], 
    net_pmu_kwargs: List[Dict] = [None],
    norm_types: List[str] = ['fstandardize'],
    seed: int = 0,
    verbose: bool = False,
    *,
    ratios=exp_log_grid(100, 3000, 100, 2),
    taus=[0.0001],
):
    """ Automates running of multiple GSP experiements. 

        Args:
            norm_types: List containing the names of normalization functions to run.
                Use the string 'raw' to apply NO normalization.
    """
    pmu = list(product(dataset_versions, difficulties, topologies, load_types, net_pmu_kwargs))
    build = list(product(norm_types, n_samples))
    exps = list(product(best_metric_names, taus))
    ###########################################################
    for dataset_version, difficulty, topology, load, net_kwargs in tqdm(pmu):
        pmuds_class = get_dataloader(dataset_type, dataset_version, difficulty)

        net_kwargs = net_kwargs if net_kwargs is not None else {}
        net_kwargs['load'] = load
        net_kwargs['topology'] = topology
        
        net_pmuds = pmuds_class(**net_kwargs).load_data()

        data_type_path = build_experiment_path(
            subdirs=[
                dataset_type,
                dataset_version,
                'gsp',
                'multi-exp',
                difficulty,
                topology,
                'simple' if net_pmuds.drop_parallel_currents else 'multi-edge',
                '-'.join(load) if isinstance(load, list) else load,
            ]
        ) 
        ################################
        for norm_type, n in tqdm(build):
            norm_fn = globals()[norm_type] if  norm_type is not None else None

            X = net_pmuds.build_graph_data(
                n=n, 
                norm_fn=norm_fn,
                random=True,
                rng=np.random.default_rng(seed=seed),
             ).graph_df
            
            save_dir_path = build_experiment_path(
                subdirs=[
                    norm_type, 
                    n if n is not None else len(net_pmuds.df),
                ],
                path=data_type_path,
            )
            print(f"Save dir path: {save_dir_path}")
            ########################################
            for select_best_metric_name, tau in exps:
                select_best_metric = metrics[select_best_metric_name]
                save_path_score = save_dir_path/select_best_metric_name
         
                single_gsp_exp(
                    X=X.T,
                    A=net_pmuds.true_network_graph,
                    ratios=ratios,
                    tau=tau,
                    metric_fn=select_best_metric['fn'],
                    select_metric=select_best_metric['type'],
                    positions=net_pmuds.graph_positions,
                    labels=net_pmuds.labels,
                    annot=True,
                    save_dir_path=save_path_score,
                    verbose=verbose,
                    disable_plotting=True
                )
                # prevent plots from being stored past saving
                plt.clf()
                plt.close('all')
   

## RTDS

### v5

In [11]:
multi_gsp_exps(
    dataset_type='rtds',
    dataset_versions=['v5'],
    difficulties=['novice'],
    taus=[0.0001],
)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/gsp/multi-exp/novice/complete/multi-edge/high/fstandardize/1802


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/gsp/multi-exp/novice/complete/multi-edge/medium/fstandardize/1802


...

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/gsp/multi-exp/novice/partial-right/multi-edge/high/fstandardize/1802


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/gsp/multi-exp/novice/partial-right/multi-edge/medium/fstandardize/1802


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/rtds/v5/gsp/multi-exp/novice/partial-right/multi-edge/low/fstandardize/1802


## MATPOWER

### Case9

In [9]:
multi_gsp_exps(
    dataset_type='matpower',
    dataset_versions=['case9'],
    difficulties=['novice'],
    topologies = ['complete'],
    load_types=['80-90', '100-110', '110-120', '80-120'],
    n_samples=[1000],
    taus=[0.0001],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case9/gsp/multi-exp/novice/complete/multi-edge/80-90/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case9/gsp/multi-exp/novice/complete/multi-edge/100-110/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case9/gsp/multi-exp/novice/complete/multi-edge/110-120/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case9/gsp/multi-exp/novice/complete/multi-edge/80-120/fstandardize/1000


### Case14

In [14]:
multi_gsp_exps(
    dataset_type='matpower',
    dataset_versions=['case14'],
    difficulties=['novice'],
    topologies = ['complete'],
    load_types=['80-90', '100-110', '110-120', '80-120'],
    n_samples=[1000],
    taus=[0.0001],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case14/gsp/multi-exp/novice/complete/multi-edge/80-90/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case14/gsp/multi-exp/novice/complete/multi-edge/100-110/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case14/gsp/multi-exp/novice/complete/multi-edge/110-120/fstandardize/1000


  0%|          | 0/1 [00:00<?, ?it/s]

Save dir path: /home/ben/space/modl/gc4eptn/exps/matpower/case14/gsp/multi-exp/novice/complete/multi-edge/80-120/fstandardize/1000
